## Equation

In [4]:
# Equation from canonical embedding
var('X,Y')
eqn = X^4 - 7/12*X^3*Y - 37/30*X^2*Y^2 + 37/30*X*Y^3 - 3/10*Y^4 - 61/60*X^3*Z + 41/15*X^2*Y*Z - 103/60*X*Y^2*Z + 19/60*Y^3*Z - 23/6*X^2*Z^2 + 87/20*X*Y*Z^2 - 14/15*Y^2*Z^2 - 199/60*X*Z^3 + 97/60*Y*Z^3 - 11/15*Z^4

## Find upper half plane representatives

- basis: ```basis_zywina_Gsq7.sage```
- Gsq7: $G_{ns}^+(p) = \langle \begin{pmatrix} 1 & 7 \\ 1 & 1 \end{pmatrix}, \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} \rangle$
- cosets: $\{T^i, (T^2)ST^i, (T^3)ST^i,(T^4)ST^i,(T^5)ST^i,(T^{12})ST^i\}$
- CM rational points discriminants = $\{ -7,-8,-11,,-19,-28,-67,-163 \}$
- CM rational points coordinaates = $\{ (3/5,2), (-2,2), (-2,-9/2), (-2, -8/3), (8/3, 2), (5/4, 2), (11, 43/2) \}$

In [1]:
load("basis_zywina_Gsq7.sage")

In [2]:
# 3 cusp forms
b0 = basislst[0]
b1 = basislst[1]
b2 = basislst[2]

In [3]:
tau7 = (1/2) + (1/2)*I*sqrt(7)
tau8 = I*sqrt(2)
tau11= (1/2) + (1/2)*I*sqrt(11)
tau19= (1/2) + (1/2)*I*sqrt(19)
tau28= I*sqrt(7)
tau67= (1/2) + (1/2)*I*sqrt(67)
tau163= (1/2) + (1/2)*I*sqrt(163)
taulist = [tau7,tau8,tau11,tau19,tau28,tau67,tau163]

In [4]:
upper_rep = [
[-7,-1/(tau7+1)+2],
[-8,-1/(tau8+12)+4],
[-11,tau11+6],
[-19,-1/(tau19+1)+3],
[-28,-1/(tau28+1)+5],
[-67,-1/(tau67+7)+5],
[-163,-1/(tau163+1)+2]] 
C1 = ComplexField(3000)
explist = [C1(e^((2*pi*I*t[1])/13)) for t in upper_rep]

In [6]:
# disc = -7 --> Q --> tQ
# disc = -11 --> R --> tR
tQ = upper_rep[0][1]
tR = upper_rep[2][1]
qQ = explist[0]
qR = explist[2]

## Compute taylor expansion

In [7]:
%%time
mfprec = 5000
R.<q> = PowerSeriesRing(QQ,mfprec)
omega0 = sum(b0[i]*q^(i-1) for i in range(mfprec))
omega1 = sum(b1[i]*q^(i-1) for i in range(mfprec))
omega2 = sum(b2[i]*q^(i-1) for i in range(mfprec))

CPU times: user 3min 8s, sys: 39.9 ms, total: 3min 8s
Wall time: 3min 8s


In [8]:
%%time
# compute the derivatives of omega_i
matsize = 50
v_list = []
for omega in [omega0,omega1,omega2]:
    for q0 in [qQ,qR]:
        #q0 = qR
        #omega = omega0
        omega_v = [omega(q=q0)]
        omega_d = omega
        for i in range(matsize-1):
            omega_d = omega_d.derivative(q)
            omega_v.append(omega_d(q=q0)/factorial(i+1))
        #v0Q = vector(omega_v)
        v_list.append(vector(omega_v))

CPU times: user 7min 12s, sys: 31.8 ms, total: 7min 12s
Wall time: 7min 12s


In [10]:
etaprec = 100
eta_1 = 1 + sum((-1)^i*(q^(13*i*(3*i-1)/2)+q^(13*i*(3*i+1)/2)) for i in range(1,etaprec))
eta_2 = 1 + sum((-1)^i*(q^(13*i*(3*i-1))+q^(13*i*(3*i+1))) for i in range(1,floor(etaprec/2)))

In [11]:
%%time
j_taylor_list = []
for q0 in [qQ,qR]:
    eta_1_taylor = eta_1(q=q0)
    eta_1_dn = eta_1
    for i in range(1,matsize):
        eta_1_dn = eta_1_dn.derivative(q)
        eta_1_taylor += eta_1_dn(q=q0)*(q^i)/factorial(i)
    eta_2_taylor = eta_2(q=q0)
    eta_2_dn = eta_2
    for i in range(1,matsize):
        eta_2_dn = eta_2_dn.derivative(q)
        eta_2_taylor += eta_2_dn(q=q0)*(q^i)/factorial(i)
    q_taylor = q0 + q

    f_taylor = (q_taylor)^13*(eta_2_taylor/eta_1_taylor)^24
    j_taylor = ((256*f_taylor + 1)^3)/f_taylor
    #j_d1_taylor = j_taylor.derivative(q)
    j_taylor_list.append(j_taylor)

CPU times: user 20min 12s, sys: 1.26 s, total: 20min 13s
Wall time: 20min 7s


In [13]:
jQ = -3375
jR = -32768
jinvariant = [jQ,jR]
jinv_list = []
djinv_list = []
for i in range(2):
    j0 = jinvariant[i]
    jinv = j_taylor_list[i] - j0
    jinv_list.append(jinv)
    j_d1_taylor = j_taylor_list[i].derivative(q)
    djinv = j_d1_taylor
    djinv_list.append(djinv)

In [14]:
%%time
M_list = []
for k in range(2):
    M = zero_matrix(C1,matsize)
    jidj = djinv_list[k].truncate_powerseries(matsize)
    for i in range(matsize):
        for j in range(matsize):
            if i <= j:
                M[j,i] = jidj[j]
        jidj = (jinv_list[k]*jidj).truncate_powerseries(matsize)
    M_list.append(M)

CPU times: user 1min 18s, sys: 36 ms, total: 1min 18s
Wall time: 1min 17s


In [15]:
sol_list = []
sol_list.append(["omega0,Q",M_list[0].solve_right(v_list[0])])
sol_list.append(["omega0,R",M_list[1].solve_right(v_list[1])])
sol_list.append(["omega1,Q",M_list[0].solve_right(v_list[2])])
sol_list.append(["omega1,R",M_list[1].solve_right(v_list[3])])
sol_list.append(["omega2,Q",M_list[0].solve_right(v_list[4])])
sol_list.append(["omega2,R",M_list[1].solve_right(v_list[5])])

In [ ]:
# Manually set the coefficients since algdep is unable to recover them as rational numbers
# Note that we have experimental estimates of the growth of the denominators

In [16]:
coeff_Q_omega0 = []
for i in range(49):
    a = (sol_list[0][1][i]*(3^(9*(i+1))*5^(3*(i+1))*13^(i+1))).real()
    #print(a)
    if str(a).count('00000000')>1:
        aa = C1(str(a).split('00000000')[0])
        #print(str(a).split('00000000')[0])
        #print(aa)
        if str(str(a).split('00000000')[-1]).count('e')>0:
            b = int(str(a).split('00000000')[-1].split('e')[-1])
        else:
            b=0
        if aa>0:
            nu = int(abs(aa*10^b))
        else:
            nu = -int(abs(aa*10^b))
    if str(a).count('9999999')>1:
        aa = str(a).split('9999999')[0]
        if aa[-1] == '.':
            b = str(int(aa[-2])+1)
            c = C1(aa[:-2]+b)
        else:
            b = str(int(aa[-1])+1)
            c = C1(aa[:-1]+b)
        if str(str(a).split('9999999')[-1]).count('e')>0:
            d = int(str(a).split('9999999')[-1].split('e')[-1])
            #print(d)
        else:
            d=0
        if c>0:
            nu = int(abs(c*10^d))
        else:
            nu = -int(abs(c*10^d))
        #print(d)
    coeff_Q_omega0.append(nu/(3^(9*(i+1))*5^(3*(i+1))*13^(i+1)))
    #print(nu)

In [17]:
coeff_Q_omega1 = []
for i in range(49):
    a = (sol_list[2][1][i]*(3^(9*(i+1))*5^(3*(i+1))*13^(i+1))).real()
    #print(a)
    if str(a).count('00000000')>1:
        aa = C1(str(a).split('00000000')[0])
        #print(str(a).split('00000000')[0])
        #print(aa)
        if str(str(a).split('00000000')[-1]).count('e')>0:
            b = int(str(a).split('00000000')[-1].split('e')[-1])
        else:
            b=0
        if aa>0:
            nu = int(abs(aa*10^b))
        else:
            nu = -int(abs(aa*10^b))
    if str(a).count('9999999')>1:
        aa = str(a).split('9999999')[0]
        if aa[-1] == '.':
            b = str(int(aa[-2])+1)
            c = C1(aa[:-2]+b)
        else:
            b = str(int(aa[-1])+1)
            c = C1(aa[:-1]+b)
        if str(str(a).split('9999999')[-1]).count('e')>0:
            d = int(str(a).split('9999999')[-1].split('e')[-1])
            #print(d)
        else:
            d=0
        if c>0:
            nu = int(abs(c*10^d))
        else:
            nu = -int(abs(c*10^d))
        #print(d)
    coeff_Q_omega1.append(nu/(3^(9*(i+1))*5^(3*(i+1))*13^(i+1)))
    #print(nu)

In [18]:
coeff_Q_omega2 = []
for i in range(49):
    a = (sol_list[4][1][i]*(3^(9*(i+1))*5^(3*(i+1))*13^(i+1))).real()
    #print(a)
    if str(a).count('00000000')>1:
        aa = C1(str(a).split('00000000')[0])
        #print(str(a).split('00000000')[0])
        #print(aa)
        if str(str(a).split('00000000')[-1]).count('e')>0:
            b = int(str(a).split('00000000')[-1].split('e')[-1])
        else:
            b=0
        if aa>0:
            nu = int(abs(aa*10^b))
        else:
            nu = -int(abs(aa*10^b))
    if str(a).count('9999999')>1:
        aa = str(a).split('9999999')[0]
        if aa[-1] == '.':
            b = str(int(aa[-2])+1)
            c = C1(aa[:-2]+b)
        else:
            b = str(int(aa[-1])+1)
            c = C1(aa[:-1]+b)
        if str(str(a).split('9999999')[-1]).count('e')>0:
            d = int(str(a).split('9999999')[-1].split('e')[-1])
            #print(d)
        else:
            d=0
        if c>0:
            nu = int(abs(c*10^d))
        else:
            nu = -int(abs(c*10^d))
        #print(d)
    coeff_Q_omega2.append(nu/(3^(9*(i+1))*5^(3*(i+1))*13^(i+1)))
    #print(nu)

In [19]:
coeff_R_omega0 = []
for i in range(50):
    a = (sol_list[1][1][i]*(2^(15*(i+1))*7^(2*(i+1))*13^(i+1))).real()
    #print(a)
    if str(a).count('00000000')>1:
        aa = C1(str(a).split('00000000')[0])
        #print(str(a).split('00000000')[0])
        #print(aa)
        if str(str(a).split('00000000')[-1]).count('e')>0:
            b = int(str(a).split('00000000')[-1].split('e')[-1])
        else:
            b=0
        if aa>0:
            nu = int(abs(aa*10^b))
        else:
            nu = -int(abs(aa*10^b))
    if str(a).count('9999999')>1:
        aa = str(a).split('9999999')[0]
        if aa[-1] == '.':
            b = str(int(aa[-2])+1)
            c = C1(aa[:-2]+b)
        else:
            b = str(int(aa[-1])+1)
            c = C1(aa[:-1]+b)
        if str(str(a).split('9999999')[-1]).count('e')>0:
            d = int(str(a).split('9999999')[-1].split('e')[-1])
            #print(d)
        else:
            d=0
        if c>0:
            nu = int(abs(c*10^d))
        else:
            nu = -int(abs(c*10^d))
        #print(d)
    coeff_R_omega0.append(nu/(2^(15*(i+1))*7^(2*(i+1))*13^(i+1)))
    #print(nu)

In [20]:
coeff_R_omega1 = []
for i in range(50):
    a = (sol_list[3][1][i]*(2^(15*(i+1))*7^(2*(i+1))*13^(i+1))).real()
    #print(a)
    if str(a).count('00000000')>1:
        aa = C1(str(a).split('00000000')[0])
        #print(str(a).split('00000000')[0])
        #print(aa)
        if str(str(a).split('00000000')[-1]).count('e')>0:
            b = int(str(a).split('00000000')[-1].split('e')[-1])
        else:
            b=0
        if aa>0:
            nu = int(abs(aa*10^b))
        else:
            nu = -int(abs(aa*10^b))
    if str(a).count('9999999')>1:
        aa = str(a).split('9999999')[0]
        if aa[-1] == '.':
            b = str(int(aa[-2])+1)
            c = C1(aa[:-2]+b)
        else:
            b = str(int(aa[-1])+1)
            c = C1(aa[:-1]+b)
        if str(str(a).split('9999999')[-1]).count('e')>0:
            d = int(str(a).split('9999999')[-1].split('e')[-1])
            #print(d)
        else:
            d=0
        if c>0:
            nu = int(abs(c*10^d))
        else:
            nu = -int(abs(c*10^d))
        #print(d)
    coeff_R_omega1.append(nu/(2^(15*(i+1))*7^(2*(i+1))*13^(i+1)))
    #print(nu)

In [21]:
coeff_R_omega2 = []
for i in range(50):
    a = (sol_list[5][1][i]*(2^(15*(i+1))*7^(2*(i+1))*13^(i+1))).real()
    #print(a)
    if str(a).count('00000000')>1:
        aa = C1(str(a).split('00000000')[0])
        #print(str(a).split('00000000')[0])
        #print(aa)
        if str(str(a).split('00000000')[-1]).count('e')>0:
            b = int(str(a).split('00000000')[-1].split('e')[-1])
        else:
            b=0
        if aa>0:
            nu = int(abs(aa*10^b))
        else:
            nu = -int(abs(aa*10^b))
    if str(a).count('9999999')>1:
        aa = str(a).split('9999999')[0]
        if aa[-1] == '.':
            b = str(int(aa[-2])+1)
            c = C1(aa[:-2]+b)
        else:
            b = str(int(aa[-1])+1)
            c = C1(aa[:-1]+b)
        if str(str(a).split('9999999')[-1]).count('e')>0:
            d = int(str(a).split('9999999')[-1].split('e')[-1])
            #print(d)
        else:
            d=0
        if c>0:
            nu = int(abs(c*10^d))
        else:
            nu = -int(abs(c*10^d))
        #print(d)
    coeff_R_omega2.append(nu/(2^(15*(i+1))*7^(2*(i+1))*13^(i+1)))
    #print(nu)

## Action of Hecke operator

In [22]:
# modular polynomial at p=11
from ast import literal_eval
with open('modpoly11.txt','r') as f:
    L = f.readlines()
modlist = [ literal_eval(i) for i in L]
R5.<x,y> = PolynomialRing(QQ)
f = 0
for i in modlist:
    if i[0][0] == i[0][1]:
        f += i[1]*x^i[0][0]*y^i[0][1]
    else:
        f += i[1]*x^i[0][0]*y^i[0][1] + i[1]*y^i[0][0]*x^i[0][1]

In [23]:
# j-invariants j(Q_i) and j(R_i) that are 11-isogeneous to Q,R respectively 
heckeimages = []
splitting_field = []
for jj in [jQ,jR]:
    fff = f(jj,y)
    R6.<y> = PolynomialRing(QQ)
    L.<d> = Rr=R6(fff).splitting_field()
    splitting_field.append(L)
    heckeimages.append(R6(fff).roots(L))

In [27]:
# account for repeated roots
heckimages_clean = []
for hecke in heckeimages:
    heckeimage_temp = []
    for i in hecke:
        heckeimage_temp += [i[0]]*i[1]
    heckimages_clean.append(heckeimage_temp)

## Tiny integrals

In [28]:
coeff_list = [coeff_Q_omega0,coeff_R_omega0,coeff_Q_omega1,coeff_R_omega1,coeff_Q_omega2,coeff_R_omega2]

In [29]:
inte_list= []
for i in range(6):
    # omega1,R
    coeff = coeff_list[i]
    #Rr.<U> = PolynomialRing(L)
    if mod(i,2) == 0:
        Rr.<U> = PolynomialRing(splitting_field[0])
        RHS = sum((coeff[i]*U^i for i in range(49)))
        RHSint = integrate(RHS,U)
        inte = sum((RHSint(U=i) for i in heckimages_clean[0]))
    else:
        Rr.<U> = PolynomialRing(splitting_field[1])
        RHS = sum((coeff[i]*U^i for i in range(49)))
        RHSint = integrate(RHS,U)
        inte = sum((RHSint(U=i) for i in heckimages_clean[1]))
    Q11 = Qp(11,30)
    inte_list.append(Q11(inte))
    #print(int)
    print(Q11(inte))

10*11^-1 + 9 + 9*11 + 6*11^2 + 7*11^3 + 9*11^4 + 8*11^5 + 4*11^6 + 7*11^7 + 11^8 + 8*11^9 + 7*11^10 + 3*11^11 + 7*11^12 + 4*11^13 + 7*11^14 + 8*11^15 + 5*11^16 + 4*11^17 + 3*11^18 + 3*11^20 + 3*11^21 + 11^22 + 9*11^24 + 11^25 + 7*11^26 + 3*11^27 + 3*11^28 + O(11^29)
7*11^-1 + 2 + 3*11 + 9*11^2 + 3*11^3 + 5*11^4 + 5*11^5 + 4*11^6 + 7*11^7 + 10*11^8 + 2*11^9 + 6*11^10 + 3*11^11 + 6*11^12 + 6*11^13 + 4*11^15 + 11^16 + 11^17 + 9*11^18 + 5*11^19 + 6*11^20 + 5*11^21 + 2*11^22 + 8*11^23 + 4*11^24 + 4*11^25 + 2*11^26 + 7*11^28 + O(11^29)
8*11^-1 + 6 + 5*11 + 8*11^2 + 4*11^3 + 5*11^4 + 9*11^5 + 3*11^6 + 9*11^7 + 5*11^8 + 4*11^10 + 10*11^11 + 9*11^12 + 9*11^13 + 4*11^14 + 7*11^15 + 3*11^16 + 5*11^17 + 3*11^18 + 9*11^20 + 11^21 + 6*11^22 + 9*11^23 + 10*11^24 + 10*11^25 + 5*11^26 + 10*11^27 + 2*11^28 + O(11^29)
6 + 6*11 + 11^3 + 5*11^4 + 7*11^5 + 10*11^6 + 5*11^7 + 3*11^9 + 11^10 + 9*11^11 + 3*11^12 + 4*11^13 + 3*11^14 + 9*11^15 + 11^16 + 5*11^18 + 10*11^19 + 7*11^20 + 3*11^21 + 9*11^22 + 10*11^24